# Set Up


In [ ]:
!nvidia-smi 
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import einops
import sage

import json
import pickle 
import copy

from tqdm.notebook import tqdm

# plotting
from functools import partial
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.renderers.default = "vscode"

# my own tooling
from utils.hook_points import HookPoint, HookedRootModule
from utils.plotting import *
from utils.groups import *
from utils.models import *
from utils.config import *

if torch.cuda.is_available:
  print('Good to go!')
else:
  print('Things might be rather slow')

In [ ]:
task_dir = "mainline-S5" #1L_MLP_sym_S5_cached_3"
seed, frac_train, layers, lr, group_param, weight_decay, num_epochs, group_type, architecture_type, metric_cfg, metric_obj = load_cfg(task_dir)
group = group_type(group_param, init_all = True)
all_data, _, all_labels, _ = generate_train_test_data(group, frac_train = 1)
model = architecture_type(layers, group.order, seed).cuda()
model.load_state_dict(torch.load(f"{task_dir}/model.pt"))
model.eval()
logits, activations = model.run_with_cache(all_data, return_cache_object=False)
activations['logits'] = logits
metric_obj = metric_obj(group, training=False, track_metrics = True)

# Logit Attribution


# Embeddings and Unembeddings

# Hidden Layer Neurons

# Logit Computation

# Ablations

# Full Circuit Analysis: Sign rep
